# Часть 2 - Использование модели машинного обучения

Добро пожаловать во вторую часть задания!
** Прежде чем продолжить, убедитесь, что сервер из первой части все еще работает.**

В этом ноутбуке напишем крохотного клиента, который использует библиотеку запросов Python для взаимодействия с вашим работающим сервером.

In [ ]:
import os
import io
import cv2
import requests
import numpy as np
from IPython.display import Image, display

## Понимание URL-адреса


### Разбивка URL-адреса

После экспериментов с клиентом fast API вы, возможно, заметили, что мы выполняли все запросы, указывая на определенный URL-адрес и добавляя к нему некоторые параметры.

Более конкретно:

1. Сервер размещен по URL-адресу [http://localhost:8000/](http://localhost:8000/).
2. Конечная точка, которая обслуживает вашу модель, - это конечная точка "/predict".

Также вы можете указать модель для использования: `yolov3` или `yolov3-tiny`. Давайте придерживаться `yolov3-tiny` для повышения вычислительной эффективности.

Давайте начнем с того, что введем в действие всю эту информацию.

In [ ]:
base_url = 'http://localhost:8000'
endpoint = '/predict'
model = 'yolov3-tiny'

Чтобы использовать вашу модель, вы добавляете конечную точку к базовому URL-адресу, чтобы получить полный URL-адрес. Обратите внимание, что параметры пока отсутствуют.

In [ ]:
url_with_endpoint_no_params = base_url + endpoint
url_with_endpoint_no_params

Чтобы задать любой из параметров, необходимо добавить символ "?", за которым следует имя параметра и его значение.

Давайте сделаем это и проверим, как выглядит конечный URL-адрес:

In [ ]:
full_url = url_with_endpoint_no_params + "?model=" + model
full_url

Эта конечная точка ожидает как имя модели, так и изображение. Но поскольку изображение более сложное, оно не передается в URL-адресе. Вместо этого мы используем библиотеку `requests` для обработки этого процесса.

# Отправка запроса на ваш сервер

### Создание функции response_from_server

Напомним, что эта конечная точка ожидает POST HTTP-запрос. Функция `post` является частью библиотеки запросов.

Чтобы передать файл вместе с запросом, вам необходимо создать словарь с указанием имени файла (в данном случае "file") и фактического файла.

`status code` - это удобная команда для проверки статуса ответа, вызванного запросом. **Код состояния 200 означает, что все прошло хорошо.**

In [ ]:
def response_from_server(url, image_file, verbose=True):
    """Отправляет POST-запрос на сервер и возвращает ответ.

    Аргументы:
        url (str): URL-адрес, на который отправляется запрос
        image_file (_io.BufferedReader): Файл для загрузки должен быть изображением.
        verbose (bool): True, если статус ответа должен быть показан. В противном случае ложь.

    Возвращает:
        requests.models.Response: Ответ от сервера.
    """
    
    files = {'file': image_file}
    response = requests.post(url, files=files)
    status_code = response.status_code
    if verbose:
        msg = "Everything went well!" if status_code == 200 else "There was an error when handling the request."
        print(msg)
    return response

Чтобы протестировать эту функцию, откройте файл в вашей файловой системе и передайте его в качестве параметра вместе с URL-адресом:

In [ ]:
with open("images/clock2.jpg", "rb") as image_file:
    prediction = response_from_server(full_url, image_file)

Отличные новости! Запрос был выполнен успешно. Однако вы не получаете никакой информации об объектах на изображении.

Чтобы получить изображение с ограничивающими рамками и метками, вам необходимо проанализировать содержимое ответа в соответствующем формате. Этот процесс очень похож на то, как вы считываете необработанные изображения в изображение cv2 на сервере.

Чтобы выполнить этот шаг, давайте создадим каталог с именем `images_predicted`, чтобы сохранить изображение в:

In [ ]:
dir_name = "images_predicted"
if not os.path.exists(dir_name):
    os.mkdir(dir_name)


### Создание функции display_image_from_response

In [ ]:
def display_image_from_response(response):
    """Отображение изображения в ответе сервера.

    Аргументы:
        response (requests.models.Response): Ответ от сервера после обнаружения объекта.
    """
    
    image_stream = io.BytesIO(response.content)
    image_stream.seek(0)
    file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
    image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    filename = "image_with_objects.jpeg"
    cv2.imwrite(f'images_predicted/{filename}', image)
    display(Image(f'images_predicted/{filename}'))

In [ ]:
display_image_from_response(prediction)

Теперь вы готовы использовать свою модель обнаружения объектов через свой собственный клиент!

Давайте проверим ее на некоторых других изображениях:

In [ ]:
image_files = [
    'car2.jpg',
    'clock3.jpg',
    'apples.jpg'
]

for image_file in image_files:
    with open(f"images/{image_file}", "rb") as image_file:
        prediction = response_from_server(full_url, image_file, verbose=False)
    
    display_image_from_response(prediction)

**Поздравляю с окончанием этой работы!** У реальных клиентов и серверов гораздо больше возможностей с точки зрения безопасности и производительности. Однако код, с которым вы только что столкнулись, близок к тому, что вы видите в реальных производственных средах.
Надеемся, что эта лабораторная работа расширила ваше представление о процессе развертывания модели глубокого обучения и использования ее в работе.

**Продолжайте в том же духе!**